## Generating a inference protobuf from Tensorflow trained model
In this example, we will take a caffe example and convert it into tensorflow model.<br>
I am using an open source library caffe-tensorflow, https://github.com/ethereon/caffe-tensorflow <br>
If tf.Session() has access to the trained graph, please skip first 6 steps

In [1]:
import sys
import os
import os.path as osp
import numpy as np
import tensorflow as tf



/wrk/hdstaff/satyakee/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Go to github work directory and clone https://github.com/ethereon/caffe-tensorflow repository

In [2]:
sys.path.insert(0, '/'.join(os.getcwd().split('/')[:-2]) + '/caffe-tensorflow')
sys.path.insert(0, '/'.join(os.getcwd().split('/')[:-2]) + '/caffe-tensorflow/examples/imagenet/')

In [3]:
from kaffe import KaffeError, print_stderr
from kaffe.tensorflow import TensorFlowTransformer

ImportError: No module named kaffe

Imports for building tensorflow graph

In [ ]:
import models
import dataset

Set outputs for the data and code for tensorflow

In [ ]:
def_path = '/'.join(os.getcwd().split('/')[:-2]) + '/caffe-tensorflow/model/bvlc_googlenet_without_lrn_deploy.prototxt'
caffemodel_path = '/'.join(os.getcwd().split('/')[:-2]) + '/caffe-tensorflow/model/bvlc_googlenet_without_lrn.caffemodel'
phase = 'test'
data_output_path = 'tf_data'
code_output_path = 'tf_out'

The following code checks if the output files are properly given or not and writes the tf code(prepares the layers) and data in to the files

In [ ]:
try:
    transformer = TensorFlowTransformer(def_path, caffemodel_path, phase=phase)
    print_stderr('Converting data...')
    if caffemodel_path is not None:
        data = transformer.transform_data()
        print_stderr('Saving data...')
        with open(data_output_path, 'wb') as data_out:
            np.save(data_out, data)
    if code_output_path:
        print_stderr('Saving source...')
        with open(code_output_path, 'wb') as src_out:
            src_out.write(transformer.transform_source())
    print_stderr('Done.')
except KaffeError as err:
    fatal_error('Error encountered: {}'.format(err))

reset the default graph in tensorflow <br>
Get the spec data from models and create the placeholder

In [ ]:
tf.reset_default_graph()

spec = models.get_data_spec(model_class=models.GoogleNet)
input_node = tf.placeholder(tf.float32,
                    shape=(None, spec.crop_size, spec.crop_size, spec.channels), name='data')

Get the image and process it

In [ ]:
fname = '/'.join(os.getcwd().split('/')[:-2]) + '/caffe-tensorflow/examples/imagenet/cat.jpg'
#    fname = 'fish-bike.jpg'

image1 = tf.image.decode_jpeg(tf.read_file(fname), channels=3)
batch1out = tf.expand_dims(image1,0)
resized1  = tf.image.resize_images(batch1out,  [spec.crop_size, spec.crop_size], tf.image.ResizeMethod.BILINEAR)

# Convert RGB to BGR
red,green,blue = tf.unstack(resized1, axis=-1)
resized2 = tf.stack([blue, green, red], axis=-1)

# Values used in CAFFE based on BGR...
IMG_MEAN = np.array((104.006989,116.66877,122.678917), dtype=np.float32)
mean_image = tf.subtract(resized2, IMG_MEAN)

In [ ]:
# Construct the network

net = models.GoogleNet({'data': input_node})

The below method creates inference graph from tensorflow trained graph and writes it to a protobuf file <br>
Also, note the train_writer writes the graph to logs in final_inference_graph. Below are the steps to see the final graph in tensorboard<br>
1. cd to current directory<br>
2. tensorboard --logdir=final_inference_graph
3. http://[server-name]:6006/

In [ ]:
# convert to list if desired...
def freeze_graph(fname, sess, outputnode):
    if type(outputnode) == tf.Tensor:
        outputname = outputnode.op.name
    else:
        outputname = outputnode
    graph_def_freeze = tf.graph_util.convert_variables_to_constants(sess, sess.graph_def, [outputname])
    graph_def = tf.graph_util.remove_training_nodes(graph_def_freeze, [outputname])
    pure_inference = tf.graph_util.extract_sub_graph(graph_def,[outputname])
    
    train_writer = tf.summary.FileWriter("final_inference_graph", sess.graph)
    train_writer.flush()
    train_writer.close()

    with tf.gfile.GFile(fname, "wb") as f:
        f.write(pure_inference.SerializeToString())
   

Testing if the model is working properly using cat.jpg results will give the probability that the image is cat or not<br><br>

Also, note the train_writer writes the graph to logs in logs. Below are the steps to see the imported graph in tensorboard<br>
1. cd to current directory<br>
2. tensorboard --logdir=final_inference_graph
3. http://[server-name]:6006/

In [ ]:
results = None
with tf.Session() as sess:
    # Load the converted parameters
    #instrumentNodes = instrument_graph(tf.get_default_graph())
    print('Loading the model')
    net.load(data_output_path, sess)
    train_writer = tf.summary.FileWriter("logs", sess.graph)
    img_data = mean_image.eval()
    #results = sess.run([net.get_output(), instrumentNodes], feed_dict={input_node: img_data})
    results = sess.run(net.get_output(), feed_dict={input_node: img_data})
    train_writer.flush()
    freeze_graph("test.pb", sess, net.get_output())

Using the results array from previous cell, display the results in human readable format

In [ ]:
def display_results(image_paths, probs):
    '''Displays the classification results given the class probability for each image'''
    # Get a list of ImageNet class labels
    with open('/'.join(os.getcwd().split('/')[:-1]) + '/caffe-tensorflow/examples/imagenet/imagenet-classes.txt', 'rb') as infile:
        class_labels = map(str.strip, infile.readlines())
    # Pick the class with the highest confidence for each image
    class_indices = np.argmax(probs, axis=1)
    # Display the results
    print('\n{:20} {:30} {}'.format('Image', 'Classified As', 'Confidence'))
    print('-' * 70)
    for img_idx, image_path in enumerate(image_paths):
        img_name = osp.basename(image_path)
        class_name = class_labels[class_indices[img_idx]]
        confidence = round(probs[img_idx, class_indices[img_idx]] * 100, 2)
        print('{:20} {:30} {} %'.format(img_name, class_name, confidence))
display_results([fname], results)